# Model Training
Load data, create sequences, train LSTM, plot loss/accuracy, predictions vs actual, confusion matrix, ROC and PR curves.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("..").resolve()))
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from data.data_loader import load_ohlcv
from src.feature_engineering import compute_all_features, FEATURE_COLUMNS
from src.data_preprocessing import drop_missing_or_fill, create_sequences, train_val_test_split_temporal, scale_features
from src.models.lstm_model import LSTMModel
from src.training.trainer import Trainer

In [ ]:
df = load_ohlcv("../data/sample_data.csv")
df = compute_all_features(df)
df = drop_missing_or_fill(df)
feat_cols = [c for c in FEATURE_COLUMNS if c in df.columns]
X = df[feat_cols].values.astype(np.float32)
close = load_ohlcv("../data/sample_data.csv")["close"].reindex(df.index).ffill()
y = (close.shift(-1) / close - 1 >= 0).astype(np.float32).values
valid = ~(np.isnan(X).any(axis=1) | np.isnan(y))
X, y = X[valid], y[valid]
X_seq, y_seq = create_sequences(X, y, 60)
(X_train, y_train), (X_val, y_val), (X_test, y_test) = train_val_test_split_temporal(X_seq, y_seq, 0.7, 0.15, 0.15)
res = scale_features(X_train, X_val, X_test)
X_train_s = res[0]
X_val_s = res[1] if len(res) > 2 else None
X_test_s = res[2] if len(res) > 3 else None
print("Train:", X_train_s.shape, y_train.shape)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size=X_train_s.shape[2], hidden_size=128, num_layers=2, dropout=0.2)
trainer = Trainer(model, device, max_epochs=30, early_stopping_patience=10)
tl, ta, vl, va = trainer.train_batched(X_train_s, y_train, X_val_s, y_val, batch_size=32)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(tl, label="Train")
plt.plot(vl, label="Val")
plt.legend()
plt.title("Loss")
plt.subplot(1, 2, 2)
plt.plot(ta, label="Train")
plt.plot(va, label="Val")
plt.legend()
plt.title("Accuracy")
plt.tight_layout()
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    pred = model(torch.tensor(X_test_s, dtype=torch.float32, device=device)).cpu().numpy().ravel()
pred_bin = (pred >= 0.5).astype(int)
from sklearn.metrics import confusion_matrix, roc_curve, auc
print("Confusion matrix:", confusion_matrix(y_test, pred_bin))
fpr, tpr, _ = roc_curve(y_test, pred)
print("AUC:", auc(fpr, tpr))
plt.figure(figsize=(5, 4))
plt.plot(fpr, tpr)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC curve")
plt.show()